In [4]:
# =========================
# AUTO-FILL FCD_id (fdcId) FROM FOOD NAMES
# =========================

import pandas as pd
import numpy as np
import requests
import time

# --- INPUTS ---
SHEET_CSV_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQwOKtoqUNXV0ag-KZsNvOureqjLz3H1BFwPoLuyEhdZi5_kvp2h-KPvc40VoziXBPjviI62Xl1oOdA/pub?output=csv"
USDA_API_KEY = "QgvQ1zpvzMyEORmbXPf5d5gTVzOONUCxzY1SR8a9"

FOOD_COL = "Food"     # from your screenshot
PRICE_COL = "Price"   # not used here, but kept
OUT_PATH = "foods_with_fcd_id.csv"

# --- USDA FDC SEARCH ---
SEARCH_URL = "https://api.nal.usda.gov/fdc/v1/foods/search"

# tweak this if you want different matching behavior
PREFERRED_DATA_TYPES = ["Foundation", "SR Legacy", "Survey (FNDDS)", "Branded"]  # priority order
PAGE_SIZE = 5
SLEEP_SEC = 0.15  # be nice to the API

def search_fdc(food_name: str, session: requests.Session):
    """
    Returns (fdcId, description, dataType) for best match, or (None, None, None)
    """
    if not food_name or not str(food_name).strip():
        return None, None, None

    payload = {
        "query": str(food_name),
        "pageSize": PAGE_SIZE,
        "pageNumber": 1,
        # you can uncomment to restrict:
        # "dataType": ["Foundation", "SR Legacy", "Survey (FNDDS)"]
    }

    r = session.post(SEARCH_URL, params={"api_key": USDA_API_KEY}, json=payload, timeout=30)
    r.raise_for_status()
    js = r.json()
    foods = js.get("foods", []) or []
    if not foods:
        return None, None, None

    # pick best match:
    # 1) first result with preferred datatype, else
    # 2) first result overall
    for dt in PREFERRED_DATA_TYPES:
        for f in foods:
            if f.get("dataType") == dt:
                return f.get("fdcId"), f.get("description"), f.get("dataType")

    f = foods[0]
    return f.get("fdcId"), f.get("description"), f.get("dataType")


# --- RUN ---
foods_df = pd.read_csv(SHEET_CSV_URL)

if FOOD_COL not in foods_df.columns:
    raise ValueError(f"Couldn't find '{FOOD_COL}' column. Columns are: {list(foods_df.columns)}")

# if FCD_id already exists, keep it; else create it
if "FCD_id" not in foods_df.columns:
    foods_df["FCD_id"] = np.nan

# add debug columns (optional but helpful)
if "FDC_match" not in foods_df.columns:
    foods_df["FDC_match"] = ""
if "FDC_type" not in foods_df.columns:
    foods_df["FDC_type"] = ""

session = requests.Session()
cache = {}  # food_name -> (fdcId, desc, dtype)

filled = 0
for i, name in enumerate(foods_df[FOOD_COL].astype(str).tolist()):
    if pd.notna(foods_df.loc[i, "FCD_id"]):
        continue  # already filled

    key = name.strip().lower()
    if key in cache:
        fdc_id, desc, dtype = cache[key]
    else:
        fdc_id, desc, dtype = search_fdc(name, session)
        cache[key] = (fdc_id, desc, dtype)
        time.sleep(SLEEP_SEC)

    if fdc_id is not None:
        foods_df.loc[i, "FCD_id"] = int(fdc_id)
        foods_df.loc[i, "FDC_match"] = desc or ""
        foods_df.loc[i, "FDC_type"] = dtype or ""
        filled += 1

print(f"✅ filled {filled} fdcIds")
missing = foods_df["FCD_id"].isna().sum()
print(f"still missing: {missing}")

foods_df.to_csv(OUT_PATH, index=False)
print("saved:", OUT_PATH)

✅ filled 176 fdcIds
still missing: 0
saved: foods_with_fcd_id.csv
